In [1]:
import numpy as np # linear algebra
import pandas as pd # 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

#Importing the auxiliar and preprocessing librarys 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import accuracy_score

In [6]:
normalized = pd.read_csv('normalized.csv', delimiter=',')
#This file contains the information of the DB without the comments of the causes 
normalized.head()

,Project,FY,Quarter,Month,Main Category,Category_details,Main_Causes,Causes_details,Amount,MonthN,CauseN
0,ALNG,2010,Q3,August,Operations,Installations,PLANNING,1.2 Late delivery from suppliers/subcontractors,1.453043,8,1
1,ALNG,2010,Q3,June,Operations,Installations,PLANNING,1.4 Ship Rescheduling/Reallocation : Change of...,7.752658,6,2
2,ALNG,2010,Q2,May,Operations,Installations,PLANNING,1.4 Ship Rescheduling/Reallocation : Change of...,7.594819,5,2
3,Block 15,2010,Q1,February,Project Management,Project Management,PLANNING,1.1 Late issue of AFC documentation,0.703771,2,3
4,Block 15,2010,Q1,December,Fabrication,Fabrication,PLANNING,1.1 Late issue of AFC documentation,0.117295,12,3


Second model: all columns

In [7]:
X = normalized.copy()

In [8]:
y = X.Amount
X.drop(['Amount'], axis=1, inplace=True)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)


In [10]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

In [11]:
# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

In [12]:
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

In [13]:
X_train.head()

,Project,Quarter,Main Category,Main_Causes,FY,MonthN,CauseN
391,Block 15,Q1,Project Management,PERFORMANCE ERRORS OR INEFFICIENCIES,2010,12,14
124,Pazflor,Q2,Operations,COMMERCIAL/CONTRACTUAL,2011,5,5
378,Block 15,Q2,Operations,PERFORMANCE ERRORS OR INEFFICIENCIES,2009,5,14
5,Block 15,Q1,Operations,PLANNING,2010,1,1
113,Pazflor,Q2,Project Management,COMMERCIAL/CONTRACTUAL,2011,5,5


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [15]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [16]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [17]:
# Define model
#model = RandomForestRegressor(n_estimators=100, random_state=0) 
model = RandomForestRegressor(n_estimators=500, random_state=0) 


In [18]:
# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [19]:
# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)


In [20]:
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 1.1034063534299374


The MAE improved from 1.410 to 1.103